<a href="https://colab.research.google.com/github/YolaYing/zk-toolkit/blob/main/Plonk%2BKZG_Implementation(Python_Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Develope Environment Preparation
Curve we used is BLS12-381
- Library we used: BLS21-381 curve implemented by Ethereum, using Python
- Lib link: https://github.com/ethereum/py_ecc/tree/main

You can use the following command to install all the packages we needed. Note that if you are using anaconda, package installation may be failed. Highly recommand using colab or some development friendly environment.

In [1]:
!pip3 install py_ecc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 872.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.6 MB/s eta 0:00:00


# KZG Implementation(Python Version)

The KZG Commitment Scheme is a commitment scheme that allows to commit to a polynomial $\Phi(x) = \phi_0 +\phi_1x+\phi_2x^2+...+\phi_lx^l$, where $\Phi(x) \in F_p[x]$ . 'to commit' means proving that you know the polynomial $\Phi(x)$ without revealing it.

The KZG commitment scheme consists of 4 steps:
1. Setup
2. Commit to Polynomials
3. Prove an Evaluation
4. Verify an Evaluation Proof

In [2]:
from py_ecc.bls12_381 import G1, G2, multiply